<a href="https://colab.research.google.com/github/JAYANTHNITW/Evolutionary_Computation_Algorithms/blob/main/BIGRU_hyperparameter_tuning_with_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras

In [2]:
df = pd.read_csv("/content/Dataset-Multitask1.csv")

In [3]:
df.head()

,Unnamed: 0,Text,Sarcasm,Stress
0,0,Peer-reviewed article that systematically deco...,0,1
1,1,Dealing with stress for children and youth liv...,0,1
2,2,"Pandemic living is stressful for everyone, eve...",0,1
3,3,"Stress leave, tears and insomnia: Greater Vict...",0,1
4,4,"Stress leave, tears and insomnia: Greater Vict...",0,1


In [4]:
df.shape

(7289, 4)

In [5]:
df = df.drop('Unnamed: 0',axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7289 entries, 0 to 7288
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     7289 non-null   object
 1   Sarcasm  7289 non-null   int64 
 2   Stress   7289 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 171.0+ KB


Tokenization

In [8]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'])

In [10]:
x = tokenizer.texts_to_sequences(df['Text'])

In [28]:
vocabulary = tokenizer.word_index
print(len(tokenizer.word_index))

6972


In [13]:
from keras.utils import pad_sequences
maxlen=25
x_paded = pad_sequences(x,maxlen=25,padding='post')

In [14]:
import gensim
!wget nlp.stanford.edu/data/glove.6B.zip

--2023-12-17 12:11:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-17 12:11:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-17 12:11:33--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [15]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [22]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [17]:
glove_input_file = "glove.6B.100d.txt"

In [18]:
glove_output_file = "word2vec.txt"

In [23]:
glove2word2vec(glove_input_file,glove_output_file)

<ipython-input-23-13f14740c424>:1: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,glove_output_file)


(400000, 100)

In [25]:
y1 = df['Sarcasm']
y2 = df['Stress']

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train_task1, y_val_task1, y_train_task2, y_val_task2 = train_test_split(
    x_paded, y1, y2, test_size=0.2, random_state=42)

In [27]:
print(f"x_train shape {x_train.shape}")
print(f"x_val shape {x_val.shape}")
print(f"y_train_task1 shape {y_train_task1.shape}")
print(f"y_train_task2 shape {y_train_task2.shape}")
print(f"y_val_task1 shape {y_val_task1.shape}")
print(f"y_val_task2 shape {y_val_task2.shape}")

x_train shape (5831, 25)
x_val shape (1458, 25)
y_train_task1 shape (5831,)
y_train_task2 shape (5831,)
y_val_task1 shape (1458,)
y_val_task2 shape (1458,)


In [29]:
print(vocabulary)

{'co': 1, 'https': 2, 't': 3, 'the': 4, 'covid': 5, '19': 6, 'to': 7, 'stress': 8, 'of': 9, 'and': 10, 'a': 11, 'in': 12, 'for': 13, 'are': 14, 'is': 15, 'with': 16, 'on': 17, 'as': 18, 'pandemic': 19, 'you': 20, 'health': 21, 'has': 22, 'relax': 23, 'at': 24, 'that': 25, 'during': 26, 'this': 27, 'have': 28, 'be': 29, 'we': 30, 'can': 31, 'it': 32, 'people': 33, 'from': 34, 'mental': 35, 'anxiety': 36, 'i': 37, 'by': 38, 'but': 39, 'our': 40, 'not': 41, 'more': 42, 'how': 43, 'their': 44, 'been': 45, 'your': 46, 'new': 47, 'amp': 48, 'about': 49, 'help': 50, 'many': 51, 'vaccine': 52, 'against': 53, 'an': 54, 'if': 55, 'some': 56, 'us': 57, 'time': 58, 'due': 59, 'teachers': 60, 'all': 61, 'need': 62, 'up': 63, 'take': 64, '…': 65, 'my': 66, 'kids': 67, 'year': 68, 'public': 69, 'or': 70, 'will': 71, 'one': 72, 'what': 73, 'than': 74, 'via': 75, 'were': 76, 'seems': 77, 'doctors': 78, 'rising': 79, 'so': 80, 'after': 81, 'get': 82, 'increased': 83, 'out': 84, 'levels': 85, 'who': 86, 

In [31]:
from gensim.models import KeyedVectors

glove_model = KeyedVectors.load_word2vec_format(glove_output_file, binary=False)

vocab_size = len(vocabulary) + 1  # Add 1 for unknown words

embedding_dim = 100  # Adjust the embedding dimension based on your GloVe model
max_length = 25
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in vocabulary.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]


In [32]:
from keras.src.layers import Concatenate
from keras.optimizers import Adam
from keras.layers import Input,Embedding,Attention,Bidirectional,GlobalMaxPool1D,GRU,Dense
from keras.models import Model
sequence_len=25
embedding_dim=100
input_shape=(sequence_len,)
input_layer = Input(shape=input_shape)
embedding_layer = Embedding(vocab_size,embedding_dim,weights=[embedding_matrix],trainable=False)(input_layer)

BiGRU_layer = Bidirectional(GRU(64,return_sequences=True))(embedding_layer)
attention_layer = Attention()([BiGRU_layer,BiGRU_layer])
concatenated_layer = Concatenate(axis=-1)([attention_layer,BiGRU_layer])
gmp_layer = GlobalMaxPool1D()(concatenated_layer)

dense_layer1 = Dense(64,activation='relu')(gmp_layer)
dense_layer2 = Dense(64,activation='relu')(gmp_layer)

output_sarcasm = Dense(1,activation='sigmoid',name='sarcasm')(dense_layer1)
concatenated_for_stress = Concatenate(axis=-1)([dense_layer2,output_sarcasm])
output_stress = Dense(1,activation='sigmoid',name='stress')(concatenated_for_stress)

model = Model(inputs=input_layer,outputs=[output_sarcasm,output_stress])
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 25, 100)              697300    ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  (None, 25, 128)              63744     ['embedding[0][0]']           
 al)                                                                                              
                                                                                                  
 attention (Attention)       (None, 25, 128)              0         ['bidirectional[0][0]',   

In [34]:
model.fit(x_train,[y_train_task1,y_train_task2],validation_data=(x_val,[y_val_task1,y_val_task2]),epochs=15,batch_size=32)

Epoch 1/15
183/183 [==============================] - 14s 40ms/step - loss: 0.9872 - sarcasm_loss: 0.4216 - stress_loss: 0.5656 - sarcasm_accuracy: 0.9329 - stress_accuracy: 0.8283 - val_loss: 0.5138 - val_sarcasm_loss: 0.1996 - val_stress_loss: 0.3142 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.8244
Epoch 2/15
183/183 [==============================] - 8s 41ms/step - loss: 0.4403 - sarcasm_loss: 0.2067 - stress_loss: 0.2337 - sarcasm_accuracy: 0.9451 - stress_accuracy: 0.9057 - val_loss: 0.3708 - val_sarcasm_loss: 0.2082 - val_stress_loss: 0.1627 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.9300
Epoch 3/15
183/183 [==============================] - 6s 32ms/step - loss: 0.4013 - sarcasm_loss: 0.2199 - stress_loss: 0.1815 - sarcasm_accuracy: 0.9408 - stress_accuracy: 0.9338 - val_loss: 0.4471 - val_sarcasm_loss: 0.2060 - val_stress_loss: 0.2411 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.9170
Epoch 4/15
183/183 [==============================] - 8s 44m

In [35]:
y_pred_task1,y_pred_task2 = model.predict(x_val)

46/46 [==============================] - 3s 10ms/step


In [36]:
threshold=0.5
y_pred_binary1 = np.where(y_pred_task1>=threshold ,1,0)
y_pred_binary2 = np.where(y_pred_task2>=threshold ,1,0)
print(y_pred_binary1)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [37]:
from sklearn.metrics import f1_score
f1_score_Sarcasm = f1_score(y_val_task1,y_pred_binary1)
print(f1_score_Sarcasm)

0.0


## HyperParameter Tuning with GA's

In [121]:
def BIGRU_model(u,n1,n2,a1,a2,op,lr):
  BiGRU_layer = Bidirectional(GRU(units=u,return_sequences=True))(embedding_layer)
  attention_layer = Attention()([BiGRU_layer,BiGRU_layer])
  concatenated_layer = Concatenate(axis=-1)([attention_layer,BiGRU_layer])
  gmp_layer = GlobalMaxPool1D()(concatenated_layer)

  dense_layer1 = Dense(units=n1,activation=a1)(gmp_layer)
  dense_layer2 = Dense(units=n2,activation=a2)(gmp_layer)

  output_sarcasm = Dense(1,activation='sigmoid',name='sarcasm')(dense_layer1)
  concatenated_for_stress = Concatenate(axis=-1)([dense_layer2,output_sarcasm])
  output_stress = Dense(1,activation='sigmoid',name='stress')(concatenated_for_stress)

  model = Model(inputs=input_layer,outputs=[output_sarcasm,output_stress])
  model.compile(loss='binary_crossentropy', optimizer=op, metrics=['accuracy'])
  model.fit(x_train,[y_train_task1,y_train_task2],validation_data=(x_val,[y_val_task1,y_val_task2]),epochs=lr,batch_size=32)


In [38]:
from random import choice,uniform
from numpy.random import randint

In [122]:
def initialization():
  parameters = {}
  u = choice([32,64,16])
  parameters["u"] = u
  n1 = choice([32,64,128])
  parameters["n1"]=n1
  n2 = choice([32,64,128])
  parameters["n2"]=n2
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  a2 = choice(["relu", "selu", "elu"])
  parameters["a2"] = a2
  op = choice(["adamax", "adadelta", "adam", "adagrad"])
  parameters["op"] = op
  lr = randint(1,30)
  parameters["lr"] = lr
  return parameters

In [ ]:
  lr = round(uniform(0.02, 0.1),2)
  parameters["lr"] = lr

In [123]:
def generate_population(n):
  population = []
  for i in range(n):
    chromosome = initialization()
    population.append(chromosome)
  return population

In [ ]:
from sklearn.metrics import accuracy_score


In [69]:
metric=model.evaluate(x_val,[y_val_task1,y_val_task2])
metric[4]

46/46 [==============================] - 1s 22ms/step - loss: 0.3513 - sarcasm_loss: 0.2078 - stress_loss: 0.1435 - sarcasm_accuracy: 0.9472 - stress_accuracy: 0.9527


0.952674925327301

In [118]:
BIGRU_model(64,16,34,'relu','relu','adam')

Epoch 1/15


KeyboardInterrupt: ignored

In [70]:
# Fitness evaluation metric: classification accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(x_val,[y_val_task1,y_val_task2])
  return metrics[4]

In [76]:
round(0.2353,2)

0.24

In [82]:
# # Roulette wheel selection method
# def selection(population_fitness,popu_size):
#   total = sum(population_fitness)
#   probilities = [round(x/total,2) for x in population_fitness]
#   multi_prob = [popu_size*prob for prob in probilities]

#   selected_parents = []
#   for index,prob in enumerate(multi_prob):
#     if round(prob,1)>1:
#       selected_parents[index] = prob
#       selected_parents[popu_size-index] = prob
#     elif round(prob,1)>=0:
#       selected_parents[index] = prob
#   return selected_parents


In [119]:
def selection(population_fitness, popu_size):
    total = sum(population_fitness)
    probabilities = [round(x / total, 2) for x in population_fitness]
    multi_prob = [int(popu_size * prob) for prob in probabilities]

    selection_wheel = []
    for index, count in enumerate(multi_prob):
        selection_wheel.extend([index] * count)
    parent1_ind = choice(selection_wheel)
    parent2_ind = choice(selection_wheel)
    return [parent1_ind, parent2_ind]

In [107]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [93]:
# fruits = ["apple", "banana", "cherry"]

# # Iterate through the list using enumerate
# for index, fruit in enumerate(fruits):
#   print(f"Fruit at index {index}: {fruit}")


In [124]:

def crossover(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["u"] = choice([parent1["u"], parent2["u"]])
  child1["n1"] = choice([parent1["n1"], parent2["n1"]])
  child1["n2"] = choice([parent1["n2"], parent2["n2"]])

  child2["u"] = choice([parent1["u"], parent2["u"]])
  child2["n1"] = choice([parent1["n1"], parent2["n1"]])
  child2["n2"] = choice([parent1["n2"], parent2["n2"]])

  child1["a1"] = parent1["a2"]
  child2["a1"] = parent2["a2"]

  child1["a2"] = parent2["a1"]
  child2["a2"] = parent1["a1"]

  child1["op"] = parent2["op"]
  child2["op"] = parent1["op"]

  child1["lr"] = parent1["lr"]
  child2["lr"] = parent2["lr"]
  return [child1, child2]

In [125]:

def mutation(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [103]:
generate_population(3)

[{'u': 32,
  'n1': 128,
  'n2': 64,
  'a1': 'relu',
  'a2': 'relu',
  'op': 'adagrad',
  'lr': 0.07},
 {'u': 16,
  'n1': 64,
  'n2': 32,
  'a1': 'relu',
  'a2': 'relu',
  'op': 'adamax',
  'lr': 0.04},
 {'u': 16,
  'n1': 64,
  'n2': 64,
  'a1': 'relu',
  'a2': 'selu',
  'op': 'adagrad',
  'lr': 0.08}]

In [126]:
generations = 3
threshold = 90
num_pop = 10

population = generate_population(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    u = chromosome["u"]
    n1 = chromosome["n1"]
    n2 = chromosome["n2"]
    a1 = chromosome["a1"]
    a2 = chromosome["a2"]
    op = chromosome["op"]
    lr = chromosome["lr"]

    try:
      model = BIGRU_model(u,n1,n2,a1,a2,op,lr)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,3))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover(parent1, parent2)
  child1 = mutation(children[0])
  child2 = mutation(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])


Epoch 1/8
183/183 [==============================] - 15s 47ms/step - loss: 1.4666 - sarcasm_loss: 0.5429 - stress_loss: 0.9238 - sarcasm_accuracy: 0.9400 - stress_accuracy: 0.1593 - val_loss: 1.3925 - val_sarcasm_loss: 0.4983 - val_stress_loss: 0.8942 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.1550
Epoch 2/8
183/183 [==============================] - 6s 33ms/step - loss: 1.3179 - sarcasm_loss: 0.4641 - stress_loss: 0.8537 - sarcasm_accuracy: 0.9451 - stress_accuracy: 0.1595 - val_loss: 1.2497 - val_sarcasm_loss: 0.4262 - val_stress_loss: 0.8235 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.1557
Epoch 3/8
183/183 [==============================] - 8s 42ms/step - loss: 1.1855 - sarcasm_loss: 0.3995 - stress_loss: 0.7860 - sarcasm_accuracy: 0.9451 - stress_accuracy: 0.1696 - val_loss: 1.1247 - val_sarcasm_loss: 0.3681 - val_stress_loss: 0.7566 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.1975
Epoch 4/8
183/183 [==============================] - 6s 33ms/st

TypeError: ignored

In [96]:
print(population)

[None, None, None, None, None, None, None, None, None, None]
